In [18]:
import json
import torch
from tqdm import tqdm

In [2]:
data_dir = '/hdd/unsupervised-summarization/data-giga/'

In [5]:
with open(data_dir+'vocab.json') as f:
    vocab = json.load(f)

In [6]:
with open(data_dir+'train_seq.json') as f:
    train = json.load(f)

In [8]:
vocab_inv = {a:b for b,a in vocab.items()}

In [10]:
def add_record(record, i):
    record['count'] += 1
    if len(i) > 0 and i[0] >= 0:
        if i[0] not in record['next']:
            record['next'][i[0]] = {'count': 0, 'next': {}}
        record['next'][i[0]] = add_record(record['next'][i[0]], i[1:])
    return record

In [11]:
lm_record = {'count':0, 'next': {}}

In [12]:
for s in tqdm(train['document']):
    s.extend([-1]*1)
    for i in range(len(s)-1):
        lm_record = add_record(lm_record, s[i:i+2])

100%|██████████| 3785004/3785004 [02:57<00:00, 21364.17it/s]


In [83]:
import numpy as np
import torch.nn.functional as F
class StatisticLM():
    def __init__(self, lm_record, vocab, lm_lambda = 0.4):
        self.lm_record = lm_record
        self.vocab = vocab
        self.lm_lambda = lm_lambda
    def forward(self, word_ids):
        output = torch.zeros((word_ids.shape[0], len(self.vocab))).float()
        for b, sen in enumerate(word_ids):
            word_list = self.lm_record['next'][sen[-1].item()]
            for (k, v) in word_list['next'].items():
                output[b, k] = v['count']
            output[b] /= word_list['count']
            for i in (output[b] == 0).nonzero():
                if i.item() in self.lm_record['next']:
                    output[b, i] = self.lm_record['next'][i.item()]['count'] / self.lm_record['count'] * self.lm_lambda
        output = F.normalize(output, p=1, dim = -1)
        return output

In [109]:
statistic_LM = StatisticLM(lm_record, vocab)

In [110]:
torch.save(statistic_LM, 'statistic_LM')